# Импорты

In [1]:
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

In [2]:
import pandas as pd
from backtesting import Backtest
from backtesting.lib import FractionalBacktest

from strategies.ma_crossover import MaCrossover
from utils.utils import time_series_split

C:\Users\MixSP\Envs\dsml\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Работа с данными

In [3]:
btc_1h_df = pd.read_csv("../data/btcusd_1hour.csv")

print(btc_1h_df.shape)
btc_1h_df.head()

(119828, 6)


,Timestamp,Open,High,Low,Close,Volume
0,2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.0
1,2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.0
2,2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.0
3,2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.0
4,2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.0


In [4]:
btc_1h_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119828 entries, 0 to 119827
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Timestamp  119828 non-null  object 
 1   Open       119828 non-null  float64
 2   High       119828 non-null  float64
 3   Low        119828 non-null  float64
 4   Close      119828 non-null  float64
 5   Volume     119828 non-null  float64
dtypes: float64(5), object(1)
memory usage: 5.5+ MB


In [5]:
# TRAIN/TEST/OOT split
btc_1h_train_df, btc_1h_test_df, btc_1h_oot_df = time_series_split(
    btc_1h_df, train_size=0.6, test_size=0.2, oot_size=0.2
)

btc_1h_train_df["Timestamp"] = pd.to_datetime(btc_1h_train_df["Timestamp"])
btc_1h_train_df.set_index("Timestamp", inplace=True)

btc_1h_test_df["Timestamp"] = pd.to_datetime(btc_1h_test_df["Timestamp"])
btc_1h_test_df.set_index("Timestamp", inplace=True)

btc_1h_oot_df["Timestamp"] = pd.to_datetime(btc_1h_oot_df["Timestamp"])
btc_1h_oot_df.set_index("Timestamp", inplace=True)

TRAIN: 2012-01-01 10:00:00 → 2020-03-15 01:00:00
TEST: 2020-03-15 02:00:00 → 2022-12-08 14:00:00
OOT: 2022-12-08 15:00:00 → 2025-09-02 23:00:00


In [6]:
btc_1h_train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71896 entries, 2012-01-01 10:00:00 to 2020-03-15 01:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    71896 non-null  float64
 1   High    71896 non-null  float64
 2   Low     71896 non-null  float64
 3   Close   71896 non-null  float64
 4   Volume  71896 non-null  float64
dtypes: float64(5)
memory usage: 3.3 MB


## Простой тест

In [7]:
bt_train = FractionalBacktest(
    btc_1h_train_df,
    fractional_unit=1e-8,
    strategy=MaCrossover,
    cash=100,
    commission=0.0018,
    margin=1.0,
    spread=0.0003,
    trade_on_close=False,
    exclusive_orders=False,
    finalize_trades=True,
)
bt_train.run()

FractionalBacktest.run:   0%|          | 0/71696 [00:00<?, ?bar/s]

Start                     2012-01-01 10:00:00
End                       2020-03-15 01:00:00
Duration                   2995 days 15:00:00
Exposure Time [%]                    99.55491
Equity Final [$]                  71176.30744
Equity Peak [$]                   95936.06814
Commissions [$]                    27390.9028
Return [%]                        71076.30744
Buy & Hold Return [%]             81384.61538
Return (Ann.) [%]                   122.52639
Volatility (Ann.) [%]                 221.748
CAGR [%]                            122.60811
Sharpe Ratio                          0.55255
Sortino Ratio                         2.17923
Calmar Ratio                          1.32963
Alpha [%]                         62019.31647
Beta                                  0.11129
Max. Drawdown [%]                   -92.15057
Avg. Drawdown [%]                    -4.82981
Max. Drawdown Duration      445 days 09:00:00
Avg. Drawdown Duration        9 days 10:00:00
# Trades                          

In [8]:
bt_test = FractionalBacktest(
    btc_1h_test_df,
    fractional_unit=1e-8,
    strategy=MaCrossover,
    cash=100,
    commission=0.0018,
    margin=1.0,
    spread=0.0003,
    trade_on_close=False,
    exclusive_orders=False,
    finalize_trades=True,
)
bt_test.run()

FractionalBacktest.run:   0%|          | 0/23765 [00:00<?, ?bar/s]

Start                     2020-03-15 02:00:00
End                       2022-12-08 14:00:00
Duration                    998 days 12:00:00
Exposure Time [%]                    98.56875
Equity Final [$]                     49.23394
Equity Peak [$]                     148.89641
Commissions [$]                      43.08757
Return [%]                          -50.76606
Buy & Hold Return [%]               185.53179
Return (Ann.) [%]                   -22.80945
Volatility (Ann.) [%]                58.79558
CAGR [%]                            -22.81945
Sharpe Ratio                         -0.38794
Sortino Ratio                         -0.4802
Calmar Ratio                         -0.32302
Alpha [%]                            -52.3643
Beta                                  0.00861
Max. Drawdown [%]                    -70.6128
Avg. Drawdown [%]                   -12.06703
Max. Drawdown Duration      563 days 18:00:00
Avg. Drawdown Duration       49 days 04:00:00
# Trades                          

In [9]:
bt_oot = FractionalBacktest(
    btc_1h_oot_df,
    fractional_unit=1e-8,
    strategy=MaCrossover,
    cash=100,
    commission=0.0018,
    margin=1.0,
    spread=0.0003,
    trade_on_close=False,
    exclusive_orders=False,
    finalize_trades=True,
)
bt_oot.run()

FractionalBacktest.run:   0%|          | 0/23767 [00:00<?, ?bar/s]

Start                     2022-12-08 15:00:00
End                       2025-09-02 23:00:00
Duration                    999 days 08:00:00
Exposure Time [%]                    99.13214
Equity Final [$]                     85.07914
Equity Peak [$]                     160.91942
Commissions [$]                      58.05016
Return [%]                          -14.92086
Buy & Hold Return [%]               561.97166
Return (Ann.) [%]                    -5.72741
Volatility (Ann.) [%]                45.23145
CAGR [%]                             -5.73112
Sharpe Ratio                         -0.12662
Sortino Ratio                        -0.18325
Calmar Ratio                         -0.10887
Alpha [%]                           -70.51448
Beta                                  0.09893
Max. Drawdown [%]                   -52.60953
Avg. Drawdown [%]                    -4.01282
Max. Drawdown Duration      532 days 06:00:00
Avg. Drawdown Duration       24 days 03:00:00
# Trades                          

## Простая оптимизация

In [10]:
stats_test = bt_test.optimize(
    ma_short_timeperiod=list(range(5, 100, 5)),
    ma_long_timeperiod=list(range(10, 500, 10)),
    ma_short_type=["SMA", "EMA"],
    ma_long_type=["SMA", "EMA"],
    maximize="Sortino Ratio",
    method="sambo",
    max_tries=1000,
    constraint=lambda param: param.ma_short_timeperiod < param.ma_long_timeperiod,
    random_state=244,
)

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

FractionalBacktest.run:   0%|          | 0/23639 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23484 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23615 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23798 [00:00<?, ?bar/s]

...

FractionalBacktest.run:   0%|          | 0/23508 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23494 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23794 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23501 [00:00<?, ?bar/s]

FractionalBacktest.run:   0%|          | 0/23492 [00:00<?, ?bar/s]

In [11]:
stats_test["_strategy"]

<Strategy MaCrossover(ma_short_timeperiod=71,ma_long_timeperiod=473,ma_short_type=EMA,ma_long_type=SMA)>

In [12]:
stats_test

Start                     2020-03-15 02:00:00
End                       2022-12-08 14:00:00
Duration                    998 days 12:00:00
Exposure Time [%]                     96.2028
Equity Final [$]                    302.16005
Equity Peak [$]                      409.4834
Commissions [$]                      54.78817
Return [%]                          202.16005
Buy & Hold Return [%]               147.18774
Return (Ann.) [%]                    49.78281
Volatility (Ann.) [%]               102.51086
CAGR [%]                             49.81312
Sharpe Ratio                          0.48563
Sortino Ratio                          1.2062
Calmar Ratio                          0.95929
Alpha [%]                           193.93558
Beta                                  0.05588
Max. Drawdown [%]                   -51.89532
Avg. Drawdown [%]                    -3.56642
Max. Drawdown Duration      654 days 15:00:00
Avg. Drawdown Duration       10 days 18:00:00
# Trades                          

In [13]:
stats_test["_strategy"]

<Strategy MaCrossover(ma_short_timeperiod=71,ma_long_timeperiod=473,ma_short_type=EMA,ma_long_type=SMA)>

In [14]:
best_params = stats_test._strategy.__dict__["_params"]
best_params

{'ma_short_timeperiod': 71,
 'ma_long_timeperiod': 473,
 'ma_short_type': 'EMA',
 'ma_long_type': 'SMA'}

In [15]:
bt_oot_fin = FractionalBacktest(
    btc_1h_oot_df,
    fractional_unit=1e-8,
    strategy=MaCrossover,
    cash=100,
    commission=0.0018,
    margin=1.0,
    spread=0.0003,
    trade_on_close=False,
    exclusive_orders=False,
    finalize_trades=True,
)

stats_oot_fin = bt_oot_fin.run(**best_params)
print(stats_oot_fin)

FractionalBacktest.run:   0%|          | 0/23494 [00:00<?, ?bar/s]

Start                     2022-12-08 15:00:00
End                       2025-09-02 23:00:00
Duration                    999 days 08:00:00
Exposure Time [%]                    97.25039
Equity Final [$]                    184.89974
Equity Peak [$]                      220.2782
Commissions [$]                      32.25588
Return [%]                           84.89974
Buy & Hold Return [%]               566.61471
Return (Ann.) [%]                    25.15026
Volatility (Ann.) [%]                59.26124
CAGR [%]                             25.16899
Sharpe Ratio                           0.4244
Sortino Ratio                         0.83516
Calmar Ratio                          0.68434
Alpha [%]                            17.39822
Beta                                  0.11913
Max. Drawdown [%]                   -36.75131
Avg. Drawdown [%]                    -3.10479
Max. Drawdown Duration      389 days 16:00:00
Avg. Drawdown Duration       13 days 12:00:00
# Trades                          

In [ ]:
bt_oot_fin.plot()

## Walk-forward оптимизация

In [17]:
print(btc_1h_train_df.shape, btc_1h_test_df.shape)
btc_1h_tt_df = pd.concat([btc_1h_train_df, btc_1h_test_df])
print(btc_1h_tt_df.shape)

(71896, 5) (23965, 5)
(95861, 5)


In [18]:
def optimize_on_window(df, strategy_cls, cash=100, commission=0.0018, spread=0.0003, random_state=244):
    bt = FractionalBacktest(
        data=df,
        fractional_unit=1e-8,
        strategy=strategy_cls,
        cash=cash,
        commission=commission,
        margin=1.0,
        spread=spread,
        trade_on_close=False,
        exclusive_orders=False,
        finalize_trades=True
    )

    stats = bt.optimize(
        ma_short_timeperiod=list(range(5, 100, 5)),
        ma_long_timeperiod=list(range(10, 500, 10)),
        ma_short_type=["SMA", "EMA"],
        ma_long_type=["SMA", "EMA"],
        maximize="Sortino Ratio",
        method="sambo",
        max_tries=500,
        constraint=lambda param: param.ma_short_timeperiod < param.ma_long_timeperiod,
        random_state=random_state,
    )

    return stats

In [19]:
def walk_forward(df, strategy_cls, train_size, test_size):
    i = 0
    n = len(df)
    start = 0
    results = []

    while start + train_size + test_size <= n:
        print('fold', i)
        train_slice = df.iloc[start : start + train_size]
        test_slice  = df.iloc[start + train_size : start + train_size + test_size]

        optimized_stat = optimize_on_window(df=train_slice, strategy_cls=strategy_cls)
        best_params = optimized_stat._strategy.__dict__["_params"]

        bt_test = FractionalBacktest(
            data=test_slice,
            fractional_unit=1e-8,
            cash=100,
            strategy=strategy_cls,
            commission=0.0018,
            margin=1.0,
            spread=0.0003,
            trade_on_close=False,
            exclusive_orders=False,
            finalize_trades=True
        )
        test_stat = bt_test.run(**best_params)

        results.append({
            "fold": i,
            "train_start": train_slice.index[0],
            "train_end": train_slice.index[-1],
            "test_start": test_slice.index[0],
            "test_end": test_slice.index[-1],
            "best_params": best_params,
            'train_stat': optimized_stat,
            'test_stat': test_stat
        })

        i += 1
        start += test_size

    return results

In [ ]:
wf_results = walk_forward(
    btc_1h_tt_df,
    MaCrossover,
    train_size=int(365 * 3 * 24), # TRAIN - 3 года
    test_size=int(365 / 1.5 * 24), # TEST - ~8 мес.
)

In [21]:
for i, fold in enumerate(wf_results, start=1):
    print(f'FOLD {i}')
    print(f'TRAIN: {fold["train_start"]} -> {fold["train_end"]} ({fold["train_end"]-fold["train_start"]})')
    print(f'TEST: {fold["test_start"]} -> {fold["test_end"]} ({fold["test_end"]-fold["test_start"]})')
    print('PARAMS:', fold['train_stat']._strategy.__dict__["_params"])
    print(f'TRAIN Sharpe:', round(fold['train_stat']['Sharpe Ratio'], 2))
    print(f'TEST Sharpe: ', round(fold['test_stat']['Sharpe Ratio'], 2))
    print(f'TRAIN Sortino:', round(fold['train_stat']['Sortino Ratio'], 2))
    print(f'TEST Sortino: ', round(fold['test_stat']['Sortino Ratio'], 2))
    print(f'TRAIN Calmar:', round(fold['train_stat']['Calmar Ratio'], 2))
    print(f'TEST Calmar: ', round(fold['test_stat']['Calmar Ratio'], 2))

    print()

FOLD 1
TRAIN: 2012-01-01 10:00:00 -> 2014-12-31 09:00:00 (1094 days 23:00:00)
TEST: 2014-12-31 10:00:00 -> 2015-08-31 17:00:00 (243 days 07:00:00)
PARAMS: {'ma_short_timeperiod': 95, 'ma_long_timeperiod': 187, 'ma_short_type': 'SMA', 'ma_long_type': 'SMA'}
TRAIN Sharpe: 0.61
TEST Sharpe:  0.93
TRAIN Sortino: 8.04
TEST Sortino:  3.82
TRAIN Calmar: 6.72
TEST Calmar:  3.69

FOLD 2
TRAIN: 2012-08-31 18:00:00 -> 2015-08-31 17:00:00 (1094 days 23:00:00)
TEST: 2015-08-31 18:00:00 -> 2016-05-01 01:00:00 (243 days 07:00:00)
PARAMS: {'ma_short_timeperiod': 95, 'ma_long_timeperiod': 185, 'ma_short_type': 'SMA', 'ma_long_type': 'SMA'}
TRAIN Sharpe: 0.69
TEST Sharpe:  0.71
TRAIN Sortino: 12.41
TEST Sortino:  2.57
TRAIN Calmar: 9.57
TEST Calmar:  3.06

FOLD 3
TRAIN: 2013-05-02 02:00:00 -> 2016-05-01 01:00:00 (1094 days 23:00:00)
TEST: 2016-05-01 02:00:00 -> 2016-12-30 09:00:00 (243 days 07:00:00)
PARAMS: {'ma_short_timeperiod': 66, 'ma_long_timeperiod': 192, 'ma_short_type': 'SMA', 'ma_long_type': '

In [22]:
best_params_wf = {
    'ma_short_timeperiod': 74,
    'ma_long_timeperiod': 471,
    'ma_short_type': 'EMA',
    'ma_long_type': 'EMA'
}

In [29]:
bt_oot_fin2 = FractionalBacktest(
    btc_1h_oot_df,
    fractional_unit=1e-8,
    strategy=MaCrossover,
    cash=100,
    commission=0.0018,
    margin=1.0,
    spread=0.0003,
    trade_on_close=False,
    exclusive_orders=False,
    finalize_trades=True,
)

# stats_oot_fin2 = bt_oot_fin2.run(**best_params_wf)
stats_oot_fin2 = bt_oot_fin2.run(**best_params)
print(stats_oot_fin2)

FractionalBacktest.run:   0%|          | 0/23494 [00:00<?, ?bar/s]

Start                     2022-12-08 15:00:00
End                       2025-09-02 23:00:00
Duration                    999 days 08:00:00
Exposure Time [%]                    97.25039
Equity Final [$]                    184.89974
Equity Peak [$]                      220.2782
Commissions [$]                      32.25588
Return [%]                           84.89974
Buy & Hold Return [%]               566.61471
Return (Ann.) [%]                    25.15026
Volatility (Ann.) [%]                59.26124
CAGR [%]                             25.16899
Sharpe Ratio                           0.4244
Sortino Ratio                         0.83516
Calmar Ratio                          0.68434
Alpha [%]                            17.39822
Beta                                  0.11913
Max. Drawdown [%]                   -36.75131
Avg. Drawdown [%]                    -3.10479
Max. Drawdown Duration      389 days 16:00:00
Avg. Drawdown Duration       13 days 12:00:00
# Trades                          

In [24]:
stats_oot_fin2_red = pd.DataFrame([stats_oot_fin2.to_dict()]).iloc[0, :-3]

In [25]:
stats_oot_fin2_red = stats_oot_fin2_red.apply(
    lambda x: round(x, 2) if isinstance(x, float) else x
)

In [26]:
stats_oot_fin2_red

Start                     2022-12-08 15:00:00
End                       2025-09-02 23:00:00
Duration                    999 days 08:00:00
Exposure Time [%]                       97.25
Equity Final [$]                        184.9
Equity Peak [$]                        220.28
Commissions [$]                         32.26
Return [%]                               84.9
Buy & Hold Return [%]                  566.61
Return (Ann.) [%]                       25.15
Volatility (Ann.) [%]                   59.26
CAGR [%]                                25.17
Sharpe Ratio                             0.42
Sortino Ratio                            0.84
Calmar Ratio                             0.68
Alpha [%]                                17.4
Beta                                     0.12
Max. Drawdown [%]                      -36.75
Avg. Drawdown [%]                        -3.1
Max. Drawdown Duration      389 days 16:00:00
Avg. Drawdown Duration       13 days 12:00:00
# Trades                          

In [27]:
bt_oot_fin2.plot()

C:\Users\MixSP\Envs\dsml\Lib\site-packages\backtesting\_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '4h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p1737', ...)